In [1]:
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification
import torch
import pandas as pd

In [2]:
tokenizer = AutoTokenizer.from_pretrained(
    "baseline_ai-forever-sbert_large_mt_nlu_ru_v2/checkpoint-530")
model = AutoModelForTokenClassification.from_pretrained(
    "baseline_ai-forever-sbert_large_mt_nlu_ru_v2/checkpoint-530")

In [43]:
import re


def most_frequent(List):
    return max(set(List), key=List.count)


def predict(text):
    text_len = len(text.split())
    inputs = tokenizer(text, return_tensors="pt",
                       max_length=512, truncation=True)
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    tokens = tokens[1:-1]

    with torch.no_grad():
        logits = model(**inputs).logits
    predictions = torch.argmax(logits, dim=2)
    predicted_token_class = [model.config.id2label[t.item()]
                             for t in predictions[0]]
    predicted_token_class = predicted_token_class[1:-1]

    convert_labels = []
    for label, token in zip(predicted_token_class, tokens):
        if token.startswith("##"):
            convert_labels[-1].append(label)
        else:
            convert_labels.append([label])

    convert_labels = [most_frequent(el) for el in convert_labels]
    convert_labels = convert_labels + ["O"] * (text_len - len(convert_labels))

    return convert_labels

In [44]:
text = "привет помните я вам предлагала скидку вот она три процента"

print(predict(text))

['O', 'O', 'O', 'O', 'O', 'B-discount', 'O', 'O', 'B-value', 'I-value']


In [45]:
test = pd.read_csv("data/gt_test.csv")
test.head()

,processed_text,label
0,здравствуйте рузан этот ээ шарна этот мы с вам...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,добрый день подскажите пожалуйста меня интерес...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,эта либергенерация чипиейная линии клиент зову...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,здравствуйте соедините пожалуйста единого счет...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,здравствуйте меня зовут NAME там еще ивакина д...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [46]:
test["label"] = test.processed_text.apply(predict)
test.head()

,processed_text,label
0,здравствуйте рузан этот ээ шарна этот мы с вам...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1,добрый день подскажите пожалуйста меня интерес...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
2,эта либергенерация чипиейная линии клиент зову...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,здравствуйте соедините пожалуйста единого счет...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,здравствуйте меня зовут NAME там еще ивакина д...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [47]:
test.to_csv("data/submission.csv", index=False)

In [48]:
test["label_set"] = test.label.apply(lambda x: "".join(list(set(x))))
test.head()

,processed_text,label,label_set
0,здравствуйте рузан этот ээ шарна этот мы с вам...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",O
1,добрый день подскажите пожалуйста меня интерес...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",I-valueB-valueB-discountO
2,эта либергенерация чипиейная линии клиент зову...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",O
3,здравствуйте соедините пожалуйста единого счет...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",O
4,здравствуйте меня зовут NAME там еще ивакина д...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",B-valueB-discountO


In [49]:
test.shape

(482, 3)

In [50]:
test[test.label_set != "O"]

,processed_text,label,label_set
1,добрый день подскажите пожалуйста меня интерес...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",I-valueB-valueB-discountO
4,здравствуйте меня зовут NAME там еще ивакина д...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",B-valueB-discountO
6,нам это двух комнатный или трех комнатный до п...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",B-valueB-discountO
11,NAME только очень быстро да а нет объект объек...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",B-discountO
13,добрый день NAME лизавета у нас пар менеджер е...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",I-valueB-valueB-discountO
...,...,...,...
471,да так сути угу да угу да верно я смотрела это...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",I-valueB-valueB-discountO
474,NAME здравствуйте служба контроля качество чег...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",B-discountO
476,NAME я вот хотел для клиента уточнить по повод...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",B-discountO
477,алло здрав вас очень плохо слыш алло я тут эду...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",I-valueB-valueB-discountO
